## Trabalho de Conclusão de Curso - TCC
#### Turma - DSA T212
#### Aluno : Drausio Gomes dos Santos
#### Orientadora: Gabrielle Maria Romeiro Lombardi
#### Tema: Construção e uso de modelos de grafos para representação e análise da estrutura organizacional da administração pública federal.
### Resultados Preliminares

### Etapa 5: Construção do Grafo Integrado
<p> Construção de um grafo integrado para fins de pesquisa de responsáveis pela assinatura de atos.</p>

In [1]:
print("Carregando pacotes necessários")
print("-----------------------------------------------")
!pip install igraph

Carregando pacotes necessários
-----------------------------------------------
Defaulting to user installation because normal site-packages is not writeable


In [2]:
print("Importando pacotes")
print("-----------------------------------------------")

#import time
import datetime as dt
from datetime import datetime
import pickle
import igraph as ig
#from igraph import *
print("igraph : ",ig.__version__)
import pandas as pd
import pytz

class Main:     
    def show_current_time(self):
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)
        return now

main = Main()
    


Importando pacotes
-----------------------------------------------
igraph :  0.10.1


###### Criando data frames integrado com todos os anos

In [5]:
  
def lista_em_maisucula(lista):
    lista_upper =[]
    for i in lista:
        lista_upper.append(i.upper())
    return lista_upper

def ajusta_dataframe(df):
    df["entidades"] = df["emissor"].str.split("/")
    df["unidade_emissora"] = df["entidades"].apply(lambda x:x[len(x)-1])
    df.insert(2,"unidade_superior", df["entidades"].apply(lambda x:x[0]))
    df['tipo'] = df['tipo'].astype('category')
    df['se_ato_assinado'] = df['responsavel'].notnull()
    df['se_possui_cargo'] = df['cargo'].notnull()
    # colocar unidades em maisucula para evitar duplicidade de vértices
    df['emissor'] = df['emissor'].apply(lambda x: x.upper())
    df['unidade_superior'] = df['unidade_superior'].apply(lambda x: x.upper())
    df['responsavel'] = df['responsavel'].apply(lambda x: str(x).upper())
    df['cargo'] = df['cargo'].apply(lambda x: str(x).upper().replace("-"," ").replace("   "," ").replace("  "," "))
    df['entidades'] = df['entidades'].apply(lambda x: lista_em_maisucula(x))
    df['unidade_emissora'] = df['unidade_emissora'].apply(lambda x: x.upper())
    return df

anos = [2019,2020,2021,2022]
base_path = "C:/projetos/tcc_dsa_usp_esalq/"
dfs = []
for ano in anos:
    anoRef = str(ano)
    data_frame_orig = pd.read_csv(base_path + "dados/dou/" + anoRef + "/" + anoRef +".csv")    
    dfs.append(ajusta_dataframe(data_frame_orig))
df = pd.concat(dfs, ignore_index=True)


#ajusta_dataframe(df)



2022 571594
2021 558114


### Análise do Dados

In [6]:
dfass = dfs[3][dfs[3].se_ato_assinado == True]
print(2022,len(dfass))
dfass = dfs[2][dfs[2].se_ato_assinado == True]
print(2021,len(dfass))
dfass = dfs[1][dfs[1].se_ato_assinado == True]
print(2020,len(dfass))
dfass = dfs[0][dfs[0].se_ato_assinado == True]
print(2019,len(dfass))

2022 571594
2021 558114
2020 471629
2019 396283


In [4]:
df_original = df.copy()
print("Checagem da consistência de ID único")
print(len(df["id"]))
print(len(df["id"].unique()))
print("-------------------------------------")
print(df["tipo"].unique())
print("-------------------------------------")
print(df.sample(100000).info())
print("-------------------------------------")
print(df.sample(100000).describe(datetime_is_numeric=True))


Checagem da consistência de ID único
3233684
3233684
-------------------------------------
['Decreto numerado' 'Lei' 'Alvará' 'Portaria' 'Despacho' 'Resolução'
 'Decisão' 'Ato' 'Ato Declaratório' 'Solução de Consulta' 'Autorização'
 'Retificação' 'Pauta' 'Ato Normativo' 'Instrução Normativa' 'Acórdão'
 'Deliberação' 'Extrato da Ata' 'Circular' 'Edital' 'Lei Complementar'
 'Mensagem' 'Portaria Conjunta' 'Ata' 'Carta Circular' 'Súmula'
 'Consulta Pública' 'Extrato de Parecer Técnico' 'Ajuste' 'Relatórios'
 'Decreto não numerado' 'Parecer Normativo' 'Agenda Regulatória'
 'Instrução' 'Balanço Patrimonial' 'Demonstração Contábil'
 'Portaria Interministerial' 'Medida Provisória' 'Decreto de Pessoal'
 'Edital de Notificação' 'Ato Concessório' 'Edital de Citação'
 'Edital de Convocação' 'Edital de Intimação' 'Lista de Antiguidade'
 'Aviso de Licitação-Concorrência' 'Aviso de Licitação-Pregão'
 'Extrato de Contrato' 'Extrato de Termo Aditivo'
 'Aviso de Homologação e Adjudicação' 'Aviso' 'Extra

In [5]:
pd.options.display.max_rows = 999
df2 = df_original.groupby(['tipo','se_ato_assinado']).size()
print(df2)

tipo                                                                             se_ato_assinado
Acordo Coletivo de Trabalho                                                      False                  65
                                                                                 True                   24
Acórdão                                                                          False                 177
                                                                                 True                 4051
Agenda Regulatória                                                               False                   3
                                                                                 True                   13
Ajuste                                                                           False                  56
                                                                                 True                   16
Alvará                                         

In [6]:
df

,id,tipo,unidade_superior,emissor,data,responsavel,cargo,entidades,unidade_emissora,se_ato_assinado,se_possui_cargo
0,S01012019/1020_20190112_991201162-2,Decreto numerado,ATOS DO PODER JUDICIÁRIO,ATOS DO PODER JUDICIÁRIO,12/01/2019,NAN,NAN,[ATOS DO PODER JUDICIÁRIO],ATOS DO PODER JUDICIÁRIO,False,False
1,S01012019/1020_20190116_11391945,Lei,ATOS DO PODER LEGISLATIVO,ATOS DO PODER LEGISLATIVO,16/01/2019,JAIR MESSIAS BOLSONARO,NAN,[ATOS DO PODER LEGISLATIVO],ATOS DO PODER LEGISLATIVO,True,False
2,S01012019/515_20190102_11303736,Alvará,MINISTÉRIO DA SEGURANÇA PÚBLICA,MINISTÉRIO DA SEGURANÇA PÚBLICA/DEPARTAMENTO D...,02/01/2019,ROSILENE GLEICE DUARTE SANTIAGO,NAN,"[MINISTÉRIO DA SEGURANÇA PÚBLICA, DEPARTAMENTO...",COORDENAÇÃO-GERAL DE CONTROLE DE SERVIÇOS E PR...,True,False
3,S01012019/515_20190102_11303780,Alvará,MINISTÉRIO DA SEGURANÇA PÚBLICA,MINISTÉRIO DA SEGURANÇA PÚBLICA/DEPARTAMENTO D...,02/01/2019,ROSILENE GLEICE DUARTE SANTIAGO,NAN,"[MINISTÉRIO DA SEGURANÇA PÚBLICA, DEPARTAMENTO...",COORDENAÇÃO-GERAL DE CONTROLE DE SERVIÇOS E PR...,True,False
4,S01012019/515_20190102_11303801,Portaria,"MINISTÉRIO DA CIÊNCIA, TECNOLOGIA, INOVAÇÕES E...","MINISTÉRIO DA CIÊNCIA, TECNOLOGIA, INOVAÇÕES E...",02/01/2019,VILMA DE FÁTIMA ALVARENGA FANIS,NAN,"[MINISTÉRIO DA CIÊNCIA, TECNOLOGIA, INOVAÇÕES ...",COORDENAÇÃO-GERAL DE RADIODIFUSÃO COMUNITÁRIA,True,False
...,...,...,...,...,...,...,...,...,...,...,...
3233679,S03112022/609_20221130_20116254,Aviso de Suspensão,MINISTÉRIO DA DEFESA,MINISTÉRIO DA DEFESA/COMANDO DA MARINHA/EMPRES...,30/11/2022,NAIR CARLA FROMENT TEIXEIRA,PREGOEIRA,"[MINISTÉRIO DA DEFESA, COMANDO DA MARINHA, EMP...",EMPRESA GERENCIAL DE PROJETOS NAVAIS,True,True
3233680,S03112022/609_20221130_20116255,Resultado de Julgamento,MINISTÉRIO DA ECONOMIA,MINISTÉRIO DA ECONOMIA/EMPRESA DE TECNOLOGIA E...,30/11/2022,MARIO HENRIQUE BERNARDO DA SILVA,PREGOEIRO,"[MINISTÉRIO DA ECONOMIA, EMPRESA DE TECNOLOGIA...",EMPRESA DE TECNOLOGIA E INFORMAÇÕES DA PREVIDÊ...,True,True
3233681,S03112022/609_20221130_20116333,Extrato de Termo Aditivo,MINISTÉRIO DAS COMUNICAÇÕES,MINISTÉRIO DAS COMUNICAÇÕES/EMPRESA BRASILEIRA...,30/11/2022,NAN,NAN,"[MINISTÉRIO DAS COMUNICAÇÕES, EMPRESA BRASILEI...",SUPERINTENDÊNCIA ESTADUAL N3 BSB,False,False
3233682,S03112022/609_20221130_20116336,Extrato de Contrato,MINISTÉRIO DAS COMUNICAÇÕES,MINISTÉRIO DAS COMUNICAÇÕES/EMPRESA BRASILEIRA...,30/11/2022,NAN,NAN,"[MINISTÉRIO DAS COMUNICAÇÕES, EMPRESA BRASILEI...",SUPERINTENDÊNCIA ESTADUAL N1 MG,False,False


#### Montando os grafo integrado utilizando IGraph

In [56]:
def cria_vertice_unidade(g,nome,label,data):
    g.add_vertices(nome)
    vert = g.vs.find(nome)            
    vert["data_inicio"]  = data
    vert["data_termino"] = data
    vert["label"] = label
    vert["weight"] =  1
    vert["tempo_medio"]=0.0 
    return vert
    
def cria_vertice_assinante(g,nome_assinante, data):    
        g.add_vertices([nome_assinante])
        vert = g.vs.find(nome_assinante)            
        vert["data_inicio"]  = data
        vert["data_termino"] = data
        vert["label"] = nome_assinante
        vert["weight"] =  1
        vert["tempo_medio"]=0.0     

def atualiza_vertice_assinante(vert_r, data):   
    if ( data < vert_r["data_inicio"] ):             
        vert_r["data_inicio"]  = data
    if ( data > vert_r["data_termino"] ):
        vert_r["data_termino"] = data
    vert_r["weight"] = vert_r["weight"] + 1
    vert_r["tempo_medio"]=(vert_r["data_termino"] - vert_r["data_inicio"]) / vert_r["weight"]

def cria_aresta_assinante(g, src, targ, data):
    g.add_edges([(src , targ)])
    eg_r = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
        edge.target_vertex["name"] == targ ) 
    if len(eg_r) > 0:
        eg_r[0]["data_inicio"] =  data
        eg_r[0]["data_termino"] =  data
        eg_r[0]["weight"] =  1
        eg_r[0]["tempo_medio"]=0.0
        eg_r[0]["label"]= str(data) + " ["+str(eg_r[0]["weight"])+"]"

    
def atualiza_aresta_assinante(g,src,targ,data):
    eg_r = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
        edge.target_vertex["name"] == targ ) 
    if len(eg_r) > 0:  
        eg_r[0]["weight"] = eg_r[0]["weight"] +  1
        if ( data < eg_r[0]["data_inicio"] ):
            eg_r[0]["data_inicio"] = data                    
        if ( data > eg_r[0]["data_termino"] ):
            eg_r[0]["data_termino"] = data
        eg_r[0]["tempo_medio"] = (eg_r[0]["data_termino"] - eg_r[0]["data_inicio"]) / eg_r[0]["weight"]
        eg_r[0]["label"]=str(eg_r[0]["data_inicio"]) + "-->" + str(eg_r[0]["data_termino"]) + " ["+str(eg_r[0]["weight"])+"]"
        if eg_r[0]["data_inicio"] is None:
            print("None")
    
def atualiza_vertice_unidade(vert, data):
    vert["weight"] = vert["weight"] +  1
    if ( data < vert["data_inicio"] ):
        vert["data_inicio"] = data                    
    if ( data > vert["data_termino"] ):
        vert["data_termino"] = data
    vert["tempo_medio"] = (vert["data_termino"] - vert["data_inicio"]) / vert["weight"]
    
def atualiza_aresta_unidade(g, src, targ, data) :
    eg = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
                            edge.target_vertex["name"] == targ )             
    if len(eg) > 0:                
        eg[0]["weight"] = eg[0]["weight"] +  1
        if ( data < eg[0]["data_inicio"] ):
            eg[0]["data_inicio"] = data                    
        if ( data > eg[0]["data_termino"] ):
            eg[0]["data_termino"] = data
        eg[0]["tempo_medio"] = (eg[0]["data_termino"] - eg[0]["data_inicio"]) / eg[0]["weight"]
        eg[0]["label"]=str(eg[0]["data_inicio"])+"-->"+str(eg[0]["data_termino"])
    else:
        g.add_edges([(src , targ)])
        eg = g.es.select(lambda edge: edge.source_vertex["name"]  == src and \
                edge.target_vertex["name"] == targ ) 
        eg[0]["data_inicio"] =  data
        eg[0]["data_termino"] =  data
        eg[0]["weight"] =  1
        eg[0]["tempo_medio"]=0.0
        eg[0]["label"]=data
    

def recupera_data(data_str):
    split_data = data_str.split("/")
    dia = split_data[0]
    mes = split_data[1]
    ano = split_data[2]
    d = dt.date(int(ano),int(mes),int(dia))
    
    return d

def recupera_tempo_em_horas(tempo):
    if 'days,' in str(tempo):
        tempo1 = str(tempo).split('days,')[0]
        tempo2 = str(tempo).split('days,')[1].split(':')        
        d,h,m,s = int(str(tempo1)), int(tempo2[0]),int(tempo2[1]),int(tempo2[2])        
    elif 'day,' in str(tempo):
        tempo1 = str(tempo).split('day,')[0]
        tempo2 = str(tempo).split('day,')[1].split(':')        
        d,h,m,s = int(str(tempo1)), int(tempo2[0]),int(tempo2[1]),int(tempo2[2])       
    else:
        tempo1 = str(tempo).split(':')       
        d,h,m,s = 0,int(tempo1[0]),int(tempo1[1]),int(tempo1[2])
        
    return d*24 + h + m/60 + s/60/60

def imprime_andamento(contador,tamanho,t0,df_mr):
    contador = contador + 1
    limite_novo_item = tamanho / 200
    if (contador % int(limite_novo_item)) == 0 :
        tempo = datetime.now(pytz.timezone('America/Sao_Paulo'))-t0
        percentual = round(contador / tamanho * 100,1)
        tempo_em_horas = recupera_tempo_em_horas(tempo)        
        df_mr.loc[len(df_mr)] = [tamanho,percentual,tempo, tempo_em_horas]     
        print(percentual, '% (' ,tempo ,') |', ' (' ,round(tempo_em_horas,6) ,' hs.) |', end=' ')        
    return contador

def associa_unidade_assinante(g, row, d, vert):
    if row["se_ato_assinado"] == True:
        id = str(row["responsavel"])+"-->"+str(row["cargo"])
        try:
            vert_r = g.vs.find(id)            
            atualiza_vertice_assinante(vert_r, d)            
            atualiza_aresta_assinante(g, vert["name"], id, d)
            
        except:
            try:
                cria_vertice_assinante(g, id, d)
                cria_aresta_assinante(g, vert["name"], id, d) 
                
            except:
                print("Erro na criação do assinante")
                
def recupera_vertice_unidade(vert_a,ent,row,d,g,index):
    if vert_a is None:
        id = ent 
    else:
        id = ent + "|" + str(vert_a["name"]) 
    try:            
        vert = g.vs.find(id)
        atualiza_vertice_unidade(vert, d)            
    except:
        vert = cria_vertice_unidade(g, id, row["entidades"][index], d)
    return vert       



In [57]:
def monta_grafo(ano, g):    
    indice = ano - anoInicial
    df = dfs[indice]   
    print("------------------------------------") 
    print("Início da criação dos grafo integrado. Ano:" + str(ano))
    print("------------------------------------") 
    t0 = main.show_current_time()
    df = df[df.se_ato_assinado == True]
    print("tamanho dataframe: ", len(df))
    df_sample = df
    print("tamanho amostra: ", len(df_sample))
    contador = 0
    tamanho = len(df_sample)
    for index, row in df_sample.iterrows():
        #print(row["entidades"])
        contador = imprime_andamento(contador, tamanho , t0, df_mr)    
        vert_a = None
        for index,ent in enumerate(row["entidades"]): 
            #print(index,ent)
            d = recupera_data(row["data"])

            vert = recupera_vertice_unidade(vert_a,ent,row,d,g,index)

            if len(row['entidades']) > index > 0:
                atualiza_aresta_unidade(g, vert_a["name"], vert["name"], recupera_data(row["data"])) 

            if index == len(row["entidades"])-1:
                associa_unidade_assinante(g, row, d, vert)

            vert_a = vert                
        if (contador % 50000) == 0:
            print(contador)
            print("Gravando dataframe intermediário." + str(contador)) 
            parte_nome_arquivo = str(anoInicial)+"_"+str(ano)+"_"+str(int(contador))
            with open("dataframe_parcial_"+ parte_nome_arquivo +".pkl", 'wb') as pickle_file:
                pickle.dump(df_mr, pickle_file)
            print("Gravando grafo intermediário." + str(contador))   
            with open("grafo_"+ parte_nome_arquivo +".pkl", 'wb') as pickle_file:
                pickle.dump(g, pickle_file)
            print(" ")   
            print("------------------------------------")   
            print(g.summary())

    print(contador)
    print("------------------------------------")            
    print("Estimativa para o dataframe completo")
    print( (datetime.now(pytz.timezone('America/Sao_Paulo'))-t0).total_seconds()/ len(df_sample) * len(df) /3600, " horas") 
    with open("grafo_"+ str(anoInicial)+"_"+str(ano)+".pkl", 'wb') as pickle_file:
        pickle.dump(g, pickle_file)
    print("Gravando dataframe final.")    
    with open("dataframe_integrado_"+ str(anoInicial)+"_"+str(ano)+".pkl", 'wb') as pickle_file:
        pickle.dump(df_mr, pickle_file)    
    tf = main.show_current_time()
    print("Término da criação dos grafo ano ."+ str(ano))    
    print("Duração:" , tf-t0)  
    print(" ")   
    print("------------------------------------")   
    print(g.summary())
    print("-----------------------------------------") 
    #for v in g.vs():
    #print(v["label"],v["weight"])
    #print(v["name"])
    #print("-------------------------------------------")
    #for e in g.es():
    #    if "-->" in e.target_vertex["label"] : #and "PAULO ROBERTO".lower() in e.target_vertex["label"].lower():
    #        print(e["label"])
    #        print(e.source_vertex["label"],e.source_vertex["weight"],e.target_vertex["label"], e.target_vertex["weight"])
    #print("-------------------------------------------")
    #for e in g.es():
    #    if "-->" not in e.target_vertex["label"] :
    #        print(e["label"],e["weight"])
    #        print(e.source_vertex["label"],e.source_vertex["weight"],e.target_vertex["label"], e.target_vertex["weight"])
    #print("-------------------------------------------")



    


In [ ]:
df_mr = pd.DataFrame([], columns=['tamanho_amostra', 'percentual_carga','tempo','tempo_em_horas'])

g = ig.Graph(directed=True)
g["name"]="Unidades Organizacionais da Administração Pública - Grafo Integrado "
anoInicial = 2019
with open("grafo_"+ str(anoInicial)+"_"+str(anoInicial-1)+".pkl", 'wb') as pickle_file:
    pickle.dump(g, pickle_file)

    
for ano in anos:
    if ano < 2021:
        continue
    g = pickle.load(open("grafo_"+ str(anoInicial)+"_"+str(ano-1)+".pkl", "rb"))
    monta_grafo(ano, g)
print("Término da criação dos grafos integrados.")  

------------------------------------
Início da criação dos grafo integrado. Ano:2021
------------------------------------
Current Time = 02/03/2023 08:59:21
tamanho dataframe:  558114
tamanho amostra:  558114
